# MNIST MLP models

#### Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets as dt, transforms

import helpers.NNUtils as nnu
from models import MLP

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1976);

#### Import data

In [3]:
train_dataset, test_dataset, val_dataset = nnu.loadMNISTDatasets()
train_loader, test_loader, val_loader = nnu.getMNISTLoaders([train_dataset, test_dataset, val_dataset],batch_size=10)

#### Train and Validate MLP models

In [4]:
model = MLP.MLPZero().to(device)
optimizer = nnu.getSGDOptim(model, lr=0.05)
loss_fn = F.cross_entropy

In [5]:
nnu.fit(5, model, loss_fn, optimizer, train_loader, test_loader)

Train Epoch: 0 	Loss: 0.246796 	Test Loss: 1.805886
Train Epoch: 1 	Loss: 0.549740 	Test Loss: 0.821152
Train Epoch: 2 	Loss: 0.486512 	Test Loss: 0.524785
Train Epoch: 3 	Loss: 0.488692 	Test Loss: 0.938117
Train Epoch: 4 	Loss: 0.467032 	Test Loss: 0.233150


In [6]:
nnu.validate(model, loss_fn, val_loader)


Validation set: Avg. loss: 0.0721, Accuracy: 0.8265 (82.7%)



In [7]:
nnu.saveModel(model, "MLPZero")

In [7]:
model = MLP.MLPOne().to(device)
optimizer = nnu.getSGDOptim(model, lr=0.05)
loss_fn = F.cross_entropy

In [8]:
nnu.fit(5, model, loss_fn, optimizer, train_loader, test_loader)

Train Epoch: 0 	Loss: 0.007551 	Test Loss: 0.184539
Train Epoch: 1 	Loss: 0.000705 	Test Loss: 0.002977
Train Epoch: 2 	Loss: 0.040968 	Test Loss: 0.001098
Train Epoch: 3 	Loss: 0.008995 	Test Loss: 0.033220
Train Epoch: 4 	Loss: 0.004902 	Test Loss: 0.030131


In [9]:
nnu.validate(model, loss_fn, val_loader)


Validation set: Avg. loss: 0.0069, Accuracy: 0.9807 (98.1%)



In [10]:
nnu.saveModel(model, "MLPOne")

'ERROR: File [./models/MLPOne] already exists!'

In [11]:
model = MLP.MLPTwo().to(device)
optimizer = nnu.getSGDOptim(model, lr=0.05)
loss_fn = F.cross_entropy

In [12]:
nnu.fit(5, model, loss_fn, optimizer, train_loader, test_loader)

Train Epoch: 0 	Loss: 0.003138 	Test Loss: 0.384565
Train Epoch: 1 	Loss: 0.052372 	Test Loss: 0.000859
Train Epoch: 2 	Loss: 0.033159 	Test Loss: 0.360157
Train Epoch: 3 	Loss: 0.000529 	Test Loss: 0.000432
Train Epoch: 4 	Loss: 0.000071 	Test Loss: 0.001945


In [13]:
nnu.validate(model, loss_fn, val_loader)


Validation set: Avg. loss: 0.0073, Accuracy: 0.9795 (98.0%)



In [14]:
nnu.saveModel(model, "MLPTwo")

'ERROR: File [./models/MLPTwo] already exists!'